# Clinical Study categorization with scispacy.

Import the things.

In [2]:
!pip install scispacy

     |████████████████████████████████| 266kB 476kB/s eta 0:00:01
     |████████████████████████████████| 10.5MB 156kB/s eta 0:00:01    |▌                               | 174kB 537kB/s eta 0:00:20
     |████████████████████████████████| 2.1MB 202kB/s eta 0:00:01
     |████████████████████████████████| 153kB 192kB/s eta 0:00:01
     |████████████████████████████████| 51kB 167kB/s eta 0:00:01
     |████████████████████████████████| 5.7MB 328kB/s eta 0:00:01
     |████████████████████████████████| 81kB 455kB/s eta 0:00:01
  ERROR: Command errored out with exit status 1:
   command: /anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/kj/qqdtl86s5kjdck2yy_27_rtw0000gn/T/pip-install-scb1e0j2/nmslib/setup.py'"'"'; __file__='"'"'/private/var/folders/kj/qqdtl86s5kjdck2yy_27_rtw0000gn/T/pip-install-scb1e0j2/nmslib/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(

Failed to build nmslib
ERROR: awsebcli 3.15.2 has requirement six<1.12.0,>=1.11.0, but you'll have six 1.12.0 which is incompatible.
ERROR: awsebcli 3.15.2 has requirement urllib3<1.25,>=1.24.1, but you'll have urllib3 1.25.3 which is incompatible.
  Running setup.py install for nmslib ... done
  Found existing installation: scikit-learn 0.20.1
    Uninstalling scikit-learn-0.20.1:
      Successfully uninstalled scikit-learn-0.20.1
  Found existing installation: botocore 1.12.199
    Uninstalling botocore-1.12.199:
      Successfully uninstalled botocore-1.12.199


In [5]:
!pip install stopwords

     |████████████████████████████████| 51kB 213kB/s eta 0:00:01
  Created wheel for stopwords: filename=stopwords-0.1.3-py2.py3-none-any.whl size=37286 sha256=269877e7b33ba8f86e2bb6827ba0881fb89a902d56c5f8b5ac47c635350dea71
  Stored in directory: /Users/connorheraty/Library/Caches/pip/wheels/39/fa/c7/c4c5111e658f5c58465d948165dc3395a3c10ff57f4cd20356
Successfully built stopwords


In [7]:
import csv
import multiprocessing
import pprint
import re
import spacy
import scispacy
import sys

from collections import defaultdict, Counter
from stopwords import STOPWORDS
from tqdm import tqdm_notebook
from multiprocessing import Pool

from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker


SyntaxError: Missing parentheses in call to 'print'. Did you mean print(lang)? (__init__.py, line 84)

In [11]:
!python -m spacy download en_core_sci_lg

/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)

✘ No compatible model found for 'en_core_sci_lg' (spaCy v2.1.4).



Set up scispacy with additional stopwords, abbreviation detection and entity detection.

In [8]:
nlp = spacy.load("en_core_sci_lg")

nlp.Defaults.stop_words |= STOPWORDS

abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)

linker = UmlsEntityLinker(resolve_abbreviations=True,
                         k=10,
                         max_entities_per_mention=3)
nlp.add_pipe(linker)

OSError: [E050] Can't find model 'en_core_sci_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

Ensure hashability for faster lookups.

In [9]:
stopwords = dict()
for word in nlp.Defaults.stop_words:
    stopwords[word] = ''

NameError: name 'nlp' is not defined

Define import functions.

In [250]:
def replace_csv_missing(row):
    """Do basic text cleaning from csv rows."""
    text = (row[0] + ' ' + 
            row[1] + ' ' + 
            row[8] + ' ' + 
            ', '.join(row[2].split()) + ', ' + 
            ', '.join(row[3].split()) + ', ' + 
            ', '.join(row[6].split()) + ', ' + 
            ', '.join(row[7].split())
           )
    new = re.sub('missing', '', text)
    new = re.sub('-', ' ', new)
    new = re.sub('\(\S*\)', ' ', new)
    new = re.sub(' \s+', ' ', new)
    new = re.sub(',,', ',', new)
    return u"{}".format(new)

In [251]:
def import_csv_files(file):
    all_rows = []
    with open(file, 'r') as csvinfile:
        csv_reader = csv.reader(csvinfile, delimiter=',')
#         for row in csv_reader:
#             all_rows.append(row)
        # Size limit for development.
        for i, row in enumerate(csv_reader):
            if i > 1000:
                break
            all_rows.append(row)
    pool = Pool(processes=multiprocessing.cpu_count())
    with pool as p:
        transformed_rows = p.map(replace_csv_missing, all_rows)
    return transformed_rows
    

Import data.

In [252]:
search_terms = import_csv_files('../all_trials_text.csv')[1:]  # Discard header row.

In [253]:
len(search_terms)

1000

## We'll need a place to store our index for fast lookup. [This](https://aws.amazon.com/blogs/aws/amazon-dynamodb-internet-scale-data-storage-the-nosql-way/) looks promising.

below is a json like idea that we should be able to upload to a db easily.

In [254]:
def make_entry(entry: str, index: int, dictionary: defaultdict) -> None:
    """Make an index entry with look-up terms as keys, tuples of document indexes and
    relevancy score as values.
    
    :var entry: text from which terms are to be extracted.
    :var index: index of text
    :var dictionary: dictionary to update with new terms and indexes.
    """
    doc = nlp(entry)
    count = Counter()
    for text in doc.ents:
        if text.lemma_.lower() not in stopwords:
            count[text.lemma_.lower()] += 1
    total = sum(count.values())
    for term, relevancy in count.items():
        dictionary[term].update({index: round(relevancy / total, 4)})
    return

In [265]:
def create_index(corpus: list) -> defaultdict:
    """Create a dict index of look-up terms.
    
    Look-up terms as keys, tuples of document indexes and
    relevancy scores as values.
    
    :var corpus: list of list of strings from which to extract look-up terms.
    
    :returns dictionary: dict of dicts "look-up term": {index: relevancy score},
    """
    dictionary = defaultdict(dict)
    for i, text in tqdm_notebook(enumerate(corpus)):
        make_entry(text, i, dictionary)
    return dictionary

In [ ]:
index = create_index(search_terms)
len(index)

In [ ]:
# Filter for display purposes. Just the entries ending in 'a'.
filtered_index = dict(filter(lambda item: item[0].endswith('a'), index.items()))
print(len(filtered_index))
pprint.pprint(filtered_index)